In [1]:
import json
import pandas as pd
import numpy as np

Jsons to get Team numbers for Opponents and URLs to get team schedule

In [3]:
with open('team_schedule.json') as f:
    data1 = json.load(f)
team_key = list(data1.keys())
team_url = list(data1.values())
f.close()

In [5]:
with open('team_numbers.json') as f:
    data = json.load(f)
team_name = list(data.keys())
team_number = list(data.values())
f.close()

Place Teams in Conferences Finals into dataframes

In [54]:
Lakers = pd.read_html(data1['LAL'],skiprows=(21,42,63))[0]
Denver = pd.read_html(data1['DEN'],skiprows=(21,42,63))[0]
Heat = pd.read_html(data1['MIA'],skiprows=(21,42,63))[0]
Boston = pd.read_html(data1['BOS'],skiprows=(21,42,63))[0]

In [55]:
Lakers = Lakers[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]
Denver = Denver[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]
Heat = Heat[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]
Boston = Boston[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]

In [56]:
list_teams = [Lakers,Denver,Heat,Boston]

Change Name to Home/Away. Change Home games to = 1 and Away values = 0

In [57]:
Lakers.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)
Denver.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)
Heat.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)
Boston.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)

In [58]:
#Can only run once or Home/Away data will say all 1s
for i in list_teams:
    list2 = []
    for index,row in i.iterrows():
        if row['Home/Away'] == '@':
            list2.append(0)
        else:
            list2.append(1)
    i['Home/Away'] = list2

Create Column for Opponent team numercial value which is determined in team_numbers.json

In [70]:
Lakers['Team_num'] = 0
Denver['Team_num'] = 0
Heat['Team_num'] = 0
Boston['Team_num'] = 0

In [71]:
for i in list_teams:
    list_ = []
    for index,row in i.iterrows():
        if row.Opponent in data:
            list_.append(data[row.Opponent])
    i['Team_num'] = list_

Create X and Y axis for each of the teams to run in the model

In [76]:
Lakers['Home'] = 16
Denver['Home'] = 18
Heat['Home'] = 3
Boston['Home'] = 5

In [77]:
Lakers_x = Lakers[["Home","Team_num","Home/Away"]].to_numpy()
Lakers_y = Lakers[["Tm","Opp"]].to_numpy()
Lakers_series = Lakers.to_numpy()

In [78]:
Denver_x = Denver[["Home","Team_num","Home/Away"]].to_numpy()
Denver_y = Denver[["Tm","Opp"]].to_numpy()
Denver_series = Denver.to_numpy()

In [80]:
Heat_x = Heat[["Home","Team_num","Home/Away"]].to_numpy()
Heat_y = Heat[["Tm","Opp"]].to_numpy()
Heat_series = Heat.to_numpy()

In [81]:
Boston_x = Boston[["Home","Team_num","Home/Away"]].to_numpy()
Boston_y = Boston[["Tm","Opp"]].to_numpy()
Boston_series = Boston.to_numpy()

Model

In [82]:
import numpy as np
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True, normalize=True)

# Train the model
model = model.fit(Lakers_x,Lakers_y)

_sq = model.score(Lakers_x, Lakers_y)

print(_sq)

0.07259547197529138


C:\Users\nehpe\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [83]:
nuggets = model.predict(np.array([[16,18,1],[16,18,0]]))
print(nuggets)

[[114.04014844 106.39708354]
 [113.37096202 108.9271338 ]]
